<a href="https://colab.research.google.com/github/Akhilez/vision_lab/blob/master/object_detection/yolo1/colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install --upgrade pytorch-lightning albumentations torchmetrics==0.5.1 wandb

     |████████████████████████████████| 1.0 MB 5.2 MB/s 
     |████████████████████████████████| 102 kB 41.2 MB/s 
     |████████████████████████████████| 282 kB 25.6 MB/s 
     |████████████████████████████████| 1.7 MB 40.9 MB/s 
     |████████████████████████████████| 596 kB 42.4 MB/s 
     |████████████████████████████████| 829 kB 45.7 MB/s 
     |████████████████████████████████| 132 kB 48.6 MB/s 
     |████████████████████████████████| 1.1 MB 40.0 MB/s 
     |████████████████████████████████| 47.6 MB 33 kB/s 
     |████████████████████████████████| 180 kB 49.4 MB/s 
     |████████████████████████████████| 97 kB 6.3 MB/s 
     |████████████████████████████████| 139 kB 45.9 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 271 kB 48.6 MB/s 
     |████████████████████████████████| 192 kB 48.4 MB/s 
     |████████████████████████████████| 160 kB 53.2 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=49107

In [2]:
! wget https://storage.googleapis.com/akhilez/datasets/mnist_detection_10k.zip
! unzip mnist_detection_10k.zip > unzip.log
! git clone https://github.com/Akhilez/vision_lab.git

--2022-02-01 21:30:10--  https://storage.googleapis.com/akhilez/datasets/mnist_detection_10k.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.188.48, 142.251.45.16, 172.217.12.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.188.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27301095 (26M) [application/zip]
Saving to: ‘mnist_detection_10k.zip’

mnist_detection_10k 100%[===================>]  26.04M  26.9MB/s    in 1.0s    

2022-02-01 21:30:11 (26.9 MB/s) - ‘mnist_detection_10k.zip’ saved [27301095/27301095]



In [3]:
%cd /content/vision_lab/

/content/vision_lab


In [4]:
from object_detection.yolo1.train_mnist_aug import train_mnist_aug_yolo1
from settings import BASE_DIR

In [5]:
architecture_config = [
    # (kernel_size, filters, stride, padding)
    (7, 64, 2, 3),  # 112 -> 56
    "M",  # 56 -> 28
    (3, 194, 1, 1),
    # [(1, 128, 1, 0), (3, 128, 1, 1), 2],
    "M",  # 28 -> 14
    [(1, 128, 1, 0), (3, 128, 1, 1), 1],
    "M",  # 14 -> 7
    [(1, 128, 1, 0), (3, 128, 1, 1), 1],
    (3, 32, 1, 0),  # 7 -> 5
]
hp = {
    "epochs": 100,
    "batch_size": 32,
    "lr_initial": 0.0001,
    "lr_decay_every": 20,
    "lr_decay_by": 0.99,
    "grid_size": 5,
    "data_augment": False,
    "num_boxes": 2,
    "lambda_coord": 5,
    "lambda_object_exists": 1,
    "lambda_no_object": 0.5,
    "lambda_class": 1,
}

config = {
    "project_name": "mnist_detection_yolo1",
    "output_path": f"{BASE_DIR}/object_detection/yolo1/output",
    "data_path_train": '/content/mnist_detection_10k/train',
    "data_path_val": '/content/mnist_detection_10k/test',
    "in_channels": 1,
    "num_classes": 10,
    "image_height": 112,
    "image_width": 112,
    "dataset_size": None,
    "num_log_images": 3,
    "dataloader_num_workers": 0,
    "num_gpus": 1,
    "architecture": architecture_config,
}

In [ ]:
train_mnist_aug_yolo1(hp, config)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 56, 56]           3,136
       BatchNorm2d-2           [-1, 64, 56, 56]             128
         LeakyReLU-3           [-1, 64, 56, 56]               0
          CNNBlock-4           [-1, 64, 56, 56]               0
         MaxPool2d-5           [-1, 64, 28, 28]               0
            Conv2d-6          [-1, 194, 28, 28]         111,744
       BatchNorm2d-7          [-1, 194, 28, 28]             388
         LeakyReLU-8          [-1, 194, 28, 28]               0
          CNNBlock-9          [-1, 194, 28, 28]               0
        MaxPool2d-10          [-1, 194, 14, 14]               0
           Conv2d-11          [-1, 128, 14, 14]          24,832
      BatchNorm2d-12          [-1, 128, 14, 14]             256
        LeakyReLU-13          [-1, 128, 14, 14]               0
         CNNBlock-14          [-1, 128,

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params
-----------------------------------------------------
0 | yolo_v1       | YoloV1             | 656 K 
1 | criterion     | YoloV1Loss         | 0     
2 | metrics_train | MyMetricCollection | 0     
3 | metrics_val   | MyMetricCollection | 0     
-----------------------------------------------------
656 K     Trainable params
0         Non-trainable params
656 K     Total params
2.626     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


Training: 0it [00:00, ?it/s]